# 基于 PyTorch Lightning 的深度学习框架代码重构和扩展
## 以 MNIST 分类和 DRIVE 分割为例

[点击查看教程](https://github.com/tobiasforest/notes)

- **作者**：林日中 `rizhonglin[at]tongji[dot]edu[dot]cn`
- **日期**：2023 年 11 月 15 日
- **地点**：上海（腾讯会议）

### 摘要

在当前的深度学习研究中，代码的可持续性和灵活性日益受到重视。本教程旨在向学生展示，仅使用 PyTorch 进行深度学习研究可能不足以应对长期和复杂的研究需求。我们通过介绍 PyTorch Lightning（PL），一个基于 PyTorch 的高级框架，来解决这一挑战。PL 作为一个通用解决方案，不仅可以简化和重构代码，还能使其更加可持续和易于维护。

此外，我们还探讨了如何针对特定领域应用高级 API 包。以医学影像分析为例，我们展示了 PL 与 MONAI（一个专注于医学影像的框架）的结合应用。通过实际案例，如 MNIST 的分类和 DRIVE 数据集的视网膜血管分割，本教程深入讲解了如何使用这些高级工具来实现更高效的数据处理、模型构建、训练和评估。

我们的目标是为学生和研究人员提供实用的见解和技能，以便他们能够在自己的研究工作中灵活运用这些高级深度学习工具。通过本教程，参与者将能够更清楚地理解 PyTorch Lightning 和 MONAI 的强大功能，以及它们在促进医学影像分析和其他深度学习任务中的重要作用。

_The images in this tutorial are sourced from the official websites of [Lightning](https://lightning.ai) and [MONAI](https://monai.io). We hereby acknowledge and appreciate the creators for their excellent work. These images are utilized solely for educational purposes, with all rights recognized as belonging to the original authors. Their contributions to education and academic research are highly valued. This tutorial is intended strictly for non-commercial use, serving as a resource for study and research._

_本教程中的图片来自 [Lightning](https://lightning.ai) 和 [MONAI](https://monai.io) 的官方网站。我们在此对创作者的出色工作表示感谢。这些图片仅用于教育目的，所有权利归原作者所有。我们高度评价他们为教育和学术研究做出的贡献。本教程仅供非商业使用，可作为学习和研究的资源。_

## Lightning AI

<img alt="Lightning" src="images/PyTorch-to-Fabric-Spectrum.png" width="2500">

### <img alt="Lightning Fabric" src="images/Lightning-Fabric.png" width="500">

Lightning Fabric 是一种快速且轻量的方法，用于扩展 PyTorch 模型，无需编写大量代码。您可以在 5 行代码内将 PyTorch 代码转换为 Lightning Fabric，从而获得对最先进的分布式训练功能（如 DDP、FSDP、DeepSpeed、混合精度等）的访问，以扩展最大的十亿参数模型。

Fabric 与 Lightning 的完全不同的几个关键方面包括：快速实现（无需重构代码），最大化灵活性（编写自己的训练和/或推理逻辑），以及最大化控制（一切都是可选的，可根据需要逐步添加更多功能）。

### <img alt="PyTorch Lightning" src="images/Pytorch-Lightning.png" width="500">

PyTorch Lightning 是一个轻量级的基于 PyTorch 的高层次模型接口，它提供了一种简单的方式来组织 PyTorch 代码，使得代码更加模块化、可读性更强、可维护性更高。它的设计目标是让研究人员专注于模型的设计，而不是训练过程的实现。

它的核心思想是将训练过程分为 5 个部分：`LightningModule`、`LightningDataModule`、`Trainer`、`Callbacks` 和 `LightningLogger`。

- `LightningModule` 是模型的核心，它包含了模型的定义、前向传播、损失函数、优化器等；
- `LightningDataModule` 是数据的核心，它包含了数据的加载、预处理、划分等；
- `Trainer` 是训练过程的核心，它包含了训练过程的超参数、优化器、学习率调整策略、训练过程的配置等；
- `Callbacks` 是训练过程的钩子函数，它包含了训练过程中的一些回调函数，如模型保存、学习率调整、训练过程可视化等；
- `LightningLogger` 是训练过程的日志记录器，它包含了训练过程中的一些日志记录，如训练过程的可视化、训练过程的日志记录等。

最新的 PyTorch Lightning 教程包括如何在 Cifar10 上训练 Resnet 以达到 94% 的准确率，如何使用 DataModules，以及如何使用 Fine-Tuning Scheduler 来微调模型。


## <img alt="MONAI" src="images/MONAI.png" width="500">

MONAI (Medical Open Network for AI) 是一个开源的医学图像分析框架，它旨在提供一个用于医学图像分析的端到端的深度学习框架，以加速 AI 在医学图像分析领域的研究和应用。

MONAI 提供了一系列的工具，如数据加载、数据预处理、数据划分、模型定义、模型训练、模型评估、模型推理等，它们都是基于 PyTorch 实现的，因此可以很方便地与 PyTorch 结合使用。MONAI 教程可以在其 GitHub 页面上找到，提供了丰富的资源和示例，以帮助用户快速上手和应用 MONAI。


### Lightning 的安装

Lightning 的安装非常简单。

若想用 pip 安装，可以使用以下命令：

```bash
pip install lightning
```

若想用 conda 安装，可以使用以下命令：

```bash
conda install lightning -c conda-forge
```

安装完成后，可以使用以下命令查看版本信息：

```bash
lightning --version
```

In [1]:
!python -c "import pytorch_lightning,lightning_fabric" || pip install lightning

### PyTorch 代码实现 MNIST 分类

下面我们以训练一个简单的 MNIST 分类模型为例，来介绍如何从 PyTorch 迁移到 Fabric 和 Lightning。

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm

# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else
                      "mps" if torch.backends.mps.is_available() else
                      "cpu")


# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the input tensor
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Define the training parameters
batch_size = 64
learning_rate = 0.01
num_epochs = 10
validation_split = 0.2

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = datasets.MNIST(root=os.getcwd(), train=True, transform=transform, download=True)

# Split the dataset into training and validation sets
train_size = int((1 - validation_split) * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = datasets.MNIST(root=os.getcwd(), train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model and move it to the device
model = Net().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch + 1}/{num_epochs}", unit="batch") as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)

            # Update the progress bar
            pbar.update(1)
            pbar.set_postfix({"Train Loss": train_loss / ((pbar.n - 1) * train_loader.batch_size + images.size(0))})

    # Validation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)

            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()

            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_loss /= len(val_loader)

    # Print validation metrics
    tqdm.write(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# Test the model
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

# Print the accuracy on the test set
accuracy = 100 * total_correct / total_samples
print(f"Test Accuracy: {accuracy:.2f}%")

# Save the model checkpoint
torch.save(model.state_dict(), "mnist.ckpt")

# Load the model checkpoint
model.load_state_dict(torch.load("mnist.ckpt"))


Epoch 1/10: 100%|██████████| 750/750 [00:04<00:00, 155.53batch/s, Train Loss=1.17]


Validation Loss: 0.5053, Validation Accuracy: 86.33%


Epoch 2/10: 100%|██████████| 750/750 [00:04<00:00, 162.17batch/s, Train Loss=0.417]


Validation Loss: 0.3741, Validation Accuracy: 89.41%


Epoch 3/10: 100%|██████████| 750/750 [00:05<00:00, 139.68batch/s, Train Loss=0.342]


...

Epoch 9/10: 100%|██████████| 750/750 [00:04<00:00, 163.51batch/s, Train Loss=0.215]


Validation Loss: 0.2336, Validation Accuracy: 93.17%


Epoch 10/10: 100%|██████████| 750/750 [00:04<00:00, 161.62batch/s, Train Loss=0.202]


Validation Loss: 0.2171, Validation Accuracy: 93.72%
Test Accuracy: 94.23%


<All keys matched successfully>

这份 PyTorch 代码实现了一个用于手写数字识别的神经网络模型。代码主要包括以下几个部分：

1. **导入所需的库**：包括 os、torch 以及 torch 的相关模块、数据集模块以及进度条模块。
2. **设置设备**：根据可用的硬件情况，将模型的计算设备设置为 CPU 或 GPU。
3. **定义神经网络模型的架构**：定义了一个包含三个全连接层的模型，输入大小为 784，输出大小为 10。
4. **定义训练参数**：包括批处理大小、学习率、训练周期数和验证集的比例。
5. **加载 MNIST 数据集**：使用 torchvision 中的 datasets 模块加载 MNIST 数据集，并进行数据预处理。
6. **将数据集划分为训练集和验证集**：使用 random_split 函数将数据集划分为训练集和验证集。
7. **创建数据加载器**：使用 DataLoader 模块创建训练集、验证集和测试集的数据加载器，用于批量加载数据。
8. **初始化模型并将其移动到设备上**：创建模型实例，并将模型移动到之前设置的设备上。
9. **定义损失函数和优化器**：使用交叉熵损失函数和随机梯度下降（SGD）优化器。
10. **训练循环**：对于每个训练周期，迭代训练集中的批次数据进行训练。计算输出、损失、梯度，然后更新模型参数。
11. **验证过程**：在每个训练周期后，使用验证集评估模型的性能。计算验证集上的损失和准确率。
12. **测试模型**：使用测试集评估模型的性能。计算测试集上的准确率。
13. **保存和加载模型**：保存训练好的模型权重参数到文件中，并可以加载之前保存的模型。

总体来说，这份代码实现了一个基于 PyTorch 的手写数字识别模型的训练、验证和测试过程，并提供了保存和加载模型的功能。

从上面的 PyTorch 代码可以看出，即使是一个简单的 MNIST 分类模型，也涉及了多个步骤，从数据处理到模型训练。尽管 PyTorch 为我们提供了强大的灵活性，但在一些场景下，代码的简洁性和扩展性可能受限。这正是 Lightning Fabric 发挥作用的地方。Fabric 作为一个新的、开源的库，使得在保持对训练循环完全控制的同时，快速且轻松地扩展模型成为可能。

Fabric 在以下几个方面优于纯 PyTorch 代码：

* **最小化更改需求**：Fabric 被引入到 PyTorch Lightning 中，旨在以最少的代码更改加速 PyTorch 的训练或推理代码，这使得它非常适合在现有的 PyTorch 项目中使用，以加速和扩展模型，而不需要大规模的重构。
* **快速实施**：与 Lightning 的完整训练器相比，Fabric 实现起来更快。不需要重新构建代码，只需在 PyTorch 脚本中更改几行代码，就可以利用 Fabric 的功能。
* **灵活性和控制**：Fabric 允许进行灵活的迭代式训练、元学习、交叉验证和其他类型的优化算法，而不需要深入了解框架内部。
* **自动化设备和精度支持**：Fabric 自动将模型和数据放置到设备上，并支持混合和双精度，从而减小内存占用。

下面我们来看看如何用 Fabric 改进这个模型。

[When to Use PyTorch Lightning or Lightning Fabric](https://lightning.ai/blog/pytorch-lightning-and-fabric)

---

尽管 Fabric 提供了一些显著的优势，比如简化代码和扩展模型的能力，但在可读性和维护性方面可能仍然存在一些挑战。为了进一步提高代码的可读性、可维护性和模块化，最终迁移到 PyTorch Lightning 就显得尤为重要。PyTorch Lightning 提供了一个更高层次的抽象，使得研究人员可以专注于模型的设计，而不是训练过程的实现，从而在代码的组织和管理上实现质的飞跃。

下面我们来看看如何用 Lightning 重构这个模型。

![PyTorch2Lightning](images/pl_quick_start_full_compressed.gif)


### LightningDataModule

首先，我们需要将数据的加载、预处理、划分等操作封装到 `LightningDataModule` 中，这样可以使得数据的加载、预处理、划分等操作更加模块化、可读性更强、可维护性更高。

In [3]:
import os
import torch
import torchvision
from torchvision import datasets, transforms
import lightning.pytorch as pl
from torch.utils.data import DataLoader, random_split


class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = os.getcwd(), batch_size: int = 64, num_workers: int = 4):
        super().__init__()
        self.data_dir = data_dir
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])
        self.batch_size = batch_size
        self.num_workers = num_workers

    def prepare_data(self):
        # Download the MNIST dataset
        datasets.MNIST(self.data_dir, train=True, download=True)
        datasets.MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        # Assign train/val/test datasets for use in dataloaders
        if stage == 'fit' or stage is None:
            mnist_full = datasets.MNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])
        if stage == 'test' or stage is None:
            self.mnist_test = datasets.MNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size, num_workers=self.num_workers,
                          persistent_workers=True, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size, num_workers=self.num_workers,
                          persistent_workers=True, shuffle=False)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size, num_workers=self.num_workers,
                          persistent_workers=True, shuffle=False)

这样一来，我们就可以将数据的加载、预处理、划分等操作从模型中分离出来，使得模型的定义更加简洁。

### LightningModule

接下来，我们需要将模型的定义、前向传播、损失函数、优化器等操作封装到 `LightningModule` 中，这样可以使得模型的定义、前向传播、损失函数、优化器等操作更加模块化、可读性更强、可维护性更高。

In [4]:
import lightning.pytorch as pl
import torch.nn as nn
import torch


# Define the neural network architecture
class MNISTModule(pl.LightningModule):
    def __init__(self, learning_rate=0.01):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        self.loss = nn.CrossEntropyLoss()
        self.lr = learning_rate

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the input tensor
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def configure_optimizers(self):
        # Define the optimizer
        optimizer = torch.optim.SGD(self.parameters(), lr=self.lr)
        return optimizer

    def training_step(self, batch, batch_idx):
        # extract x and y from the batch
        x, y = batch

        # forward pass
        logits = self(x)

        # calculate the loss
        loss = self.loss(logits, y)

        # Logging to the built-in logger
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        # extract x and y from the batch
        x, y = batch

        # forward pass
        logits = self(x)

        # calculate the loss
        loss = self.loss(logits, y)

        # calculate the accuracy
        _, predicted = torch.max(logits.data, 1)
        correct = (predicted == y).sum().item()

        # Logging to the built-in logger
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', correct / len(y), prog_bar=True)

        return loss

    def test_step(self, batch, batch_idx):
        # extract x and y from the batch
        x, y = batch

        # forward pass
        logits = self(x)

        # calculate the loss
        loss = self.loss(logits, y)

        # calculate the accuracy
        _, predicted = torch.max(logits.data, 1)
        correct = (predicted == y).sum().item()

        # Logging to the built-in logger
        self.log('test_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('test_acc', correct / len(y), on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return {'test_loss': loss, 'correct': correct, 'total': len(y)}


### Trainer

在定义好 `LightningDataModule` 和 `LightningModule` 后，我们就可以使用 `Trainer` 来训练模型了。

`Trainer` 是 Lightning 提供的一个用于训练模型的类，它封装了训练过程中的大量细节，使得训练过程更加简单。

In [5]:
# Initialize data module and model
data_module = MNISTDataModule()
model = MNISTModule()

callbacks = [
    # Early stopping callback to prevent overfitting
    pl.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='min'),
    # Model checkpoint callback to save the best model(s)
    pl.callbacks.ModelCheckpoint(monitor='val_loss')
]

# Initialize trainer
trainer = pl.Trainer(
    max_epochs=20,  # Number of epochs, if -1, runs indefinitely
    precision='16-mixed',  # precision of training, default is 32-bit
    callbacks=callbacks,  # callbacks defined above
    accelerator='cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    # device to use for training
)

# Tune the model hyperparameters
from lightning.pytorch.tuner import Tuner

tuner = Tuner(trainer)

# # Find the optimal batch size that maximizes GPU utilization
# tuner.scale_batch_size(model, data_module)

# Find the "optimal" learning rate (BE CAREFUL)
tuner.lr_find(model, data_module)

# Fit the model
trainer.fit(model, data_module)

Using 16bit Automatic Mixed Precision (AMP)
/Users/rizhong/miniconda3/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:124: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/rizhong/miniconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
/Users/rizhong/miniconda3/lib/python3.11/site-packages/torch/amp/auto

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.47863009232263803
Restoring states from the checkpoint path at /Users/rizhong/Documents/notes/20231115_LightningMONAITutorial/.lr_find_a4c87f42-d9bf-4a4a-b7dd-49412cb10e9a.ckpt
Restored all states from the checkpoint at /Users/rizhong/Documents/notes/20231115_LightningMONAITutorial/.lr_find_a4c87f42-d9bf-4a4a-b7dd-49412cb10e9a.ckpt

  | Name | Type             | Params
------------------------------------------
0 | fc1  | Linear           | 100 K 
1 | fc2  | Linear           | 8.3 K 
2 | fc3  | Linear           | 650   
3 | loss | CrossEntropyLoss | 0     
------------------------------------------
109 K     Trainable params
0         Non-trainable params
109 K     Total params
0.438     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [6]:
# Test the model
trainer.test(model, data_module)

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.9645000100135803     │
│      test_loss_epoch      │    0.19310082495212555    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 0.19310082495212555,
  'test_acc_epoch': 0.9645000100135803}]

这种方式使用 PyTorch Lightning 框架可以大幅简化代码，提高模型的可读性和可维护性，同时保持了灵活性和强大的功能。

### MONAI 的安装

MONAI 的安装非常简单。

若想用 pip 安装，可以使用以下命令：

```bash
pip install monai
```

若想用 conda 安装，可以使用以下命令：

```bash
conda install monai -c conda-forge
```

安装完成后，可以使用以下命令查看版本信息：

```bash
python -c "import monai; print(monai.__version__)"
```

In [7]:
!python -c "import monai" || pip install 'monai[all]'

### MONAI 为什么值得关注

MONAI（Medical Open Network for AI）是专门为医学影像研究而设计的深度学习框架。它基于 PyTorch，提供了一系列针对医学影像处理的优化工具和方法。以下是一些让 MONAI 引人注目的特点：

1. **多维医学影像数据的灵活预处理**：MONAI 提供了一系列预处理和增强工具，这些工具可以处理不同维度和不同模态（如 CT、MRI）的医学影像数据。这大大提高了数据的可用性和质量。
2. **组合式和可移植的 API**：MONAI 设计了简洁而强大的 API，使得它可以轻松地集成到现有的医学影像处理和分析工作流程中。这种组合式的方法提供了极高的灵活性，允许研究者根据需要自定义流程。
3. **针对特定领域的网络、损失函数和评估指标实现**：MONAI 提供了一系列专为医学影像分析定制的网络架构、损失函数和评估指标。这些实现考虑到了医学影像数据的特殊性，如不同类型的数据不平衡、高维度和高解析度等问题。
4. **与现有的医学影像分析工具兼容**：MONAI 可以与其他流行的医学影像工具（如 ITK、SimpleITK、DICOM、NiBabel 等）无缝集成，方便从事此领域研究的科学家和工程师们导入和导出数据。

<img alt="MONAI Architecture" src="images/MONAI_arch_modules.png" width="1000">

#### 医学影像数据预处理方法

在 MONAI 中，数据处理方法（也称为 transform）是处理医学影像数据的核心组成部分。这些方法旨在提高医学影像数据的质量和可用性，从而提升模型训练和评估的效果。MONAI 提供了一系列基于字典的预处理方法，为处理复杂的医学影像数据提供了强大的灵活性和便利性。以下是一些常用的基于字典的预处理方法：

- `LoadImaged`：加载影像数据。能够处理各种不同格式的医学影像，如 NIFTI、DICOM，以及标准的图像格式，如 PNG、JPG 等。
- `EnsureChannelFirstd`：确保影像数据的通道维度位于第一位，符合 PyTorch 的默认通道顺序。
- `MaskIntensityd`：根据给定的遮罩，将影像数据中的非感兴趣区域置为指定的值，以提高模型的训练效果。
- `AsDiscreted`：将影像数据转换为离散的 One-Hot 编码，适用于分类任务。
- `Orientationd`：调整影像数据的方向，使其符合标准方向，解决由于设备差异导致的方向不一致问题。
- `Resized`：调整影像的大小，实现输入数据尺寸的统一。
- `NormalizeIntensityd`：标准化影像的强度值，消除设备间的强度差异。
- `RandRotate90d`：随机旋转影像，增强模型从不同角度识别图像的能力。
- `RandFlipd`：随机翻转影像，增强模型对影像方向变化的鲁棒性。
- `RandGaussianNoised`：向影像中添加随机高斯噪声，提升模型处理噪声数据的能力。
- `RandGaussianSmoothd`：对影像进行随机高斯平滑处理，提升模型处理噪声数据的能力。
- `RandScaleIntensityd`：对影像进行随机强度缩放，提升模型处理不同强度数据的能力。
- `Rand2DElasticd`：对影像进行随机二维弹性变形，提升模型处理不同形变数据的能力。
- `CropForegroundd`：剪裁掉影像中的前景部分，专注于特定结构或区域的分析。
- `RandCropByPosNegLabeld`：根据正负标签随机剪裁影像，平衡正负样本。
- `ToTensord`：将影像转换为 PyTorch 张量，为深度学习做准备。
- `Lambdad`：允许自定义数据处理方法，增加数据处理的灵活性。
- `Compose`：将多个数据处理方法组合起来，构建数据处理流程。

这些预处理方法可以灵活配置和组合，以针对特定医学影像任务构建有效的数据处理流程。在 MONAI 中，这些方法通常以字典形式传递，使得每个处理步骤都可以访问到数据集中的相关信息，如影像路径、标签、元数据等，增加处理过程的透明度和可控性。

#### 数据加载

在 MONAI 中，数据加载是指将医学影像数据加载到内存中，以便后续的预处理和模型训练。

除了常规的 `Dataset` 外，MONAI 提供了 `CacheDataset` 和 `SmartCacheDataset` 两种数据集，用于优化大规模医学影像数据的加载过程。

- `CacheDataset`：预先加载并处理整个数据集，然后将其存储在内存中。这对于相对较小的数据集非常有效，因为它减少了每个 epoch 时的计算开销。
- `SmartCacheDataset`：仅预加载和处理数据集的一部分，并在训练过程中逐渐替换旧的数据项。这对于大型数据集特别有用，因为它减少了初始加载时间，并且可以动态调整数据集的内容。

#### 模型构建

MONAI 提供了多种内置模型，特别是针对医学影像任务设计的模型，如 UNet、VNet、DenseNet、HighResNet 等。这些模型可以直接用于医学影像分析任务，也可以用作预训练模型的基础。

#### 模型验证

在处理尺寸较大的医学影像数据时，我们通常会使用前面提到的 `RandCropByPosNegLabeld` 等方法对数据进行剪裁，以减少内存占用。

在模型验证阶段，MONAI 的 `SlidingWindowInferer` 可以用于处理大尺寸的医学影像。它将大影像切分成小窗口，逐个送入模型进行推理，然后将结果拼接回完整的影像。这种方法特别适用于 3D 影像或大尺寸的 2D 影像。

#### 模型评估

MONAI 支持多种损失函数和评估指标，适用于不同的医学影像分析任务：

- 损失函数：Dice 损失、交叉熵损失、Focal 损失等，这些损失函数可以帮助处理医学影像中常见的类别不平衡问题。
- 评估指标：Dice 系数、灵敏度、特异性等，这些指标用于量化模型在医学影像分析任务上的性能。



通过结合使用这些高效的数据加载方法、强大的模型架构、灵活的推理工具以及有效的损失函数和评估指标，MONAI 能够在医学影像分析领域提供出色的性能和灵活性。

### 实现一个简单的医学影像分割模型

我们将结合 MONAI 和 PyTorch Lightning 来训练一个用于分割视网膜血管的模型。我们选用的是 DRIVE（Digital Retinal Images for Vessel Extraction）数据集，这是一个广泛使用的医学影像数据集，专门用于视网膜血管分割任务。该数据集包含 20 个训练样本和 20 个测试样本，每个样本包含以下 3 个文件：

1. **视网膜图像**：彩色的眼底照片，用于作为模型的输入。
2. **血管分割图像**：标记了视网膜血管的图像，用作训练和评估模型时的真值（ground truth）。
3. **视盘分割图像**：标记了视网膜视盘区域的图像，有时用于提高分割任务的准确性。

#### 数据预处理

首先，我们需要对 DRIVE 数据集进行预处理。使用 MONAI 的预处理功能，我们可以轻松地加载、标准化和转换这些图像，使其适合用于深度学习模型。预处理步骤可能包括：

- 转换彩色视网膜图像为灰度图像。
- 标准化图像的尺寸和强度值。
- 应用数据增强，如旋转、翻转、弹性变形等。

In [8]:
import os
import lightning.pytorch as pl
from torch.utils.data import random_split
from monai.data import CacheDataset, DataLoader
from monai.transforms import (
    Compose,
    LoadImaged,
    MaskIntensityd,
    EnsureChannelFirstd,
    NormalizeIntensityd,
    ScaleIntensityd,
    AsDiscreted,
    Resized,
    RandRotate90d,
    RandFlipd,
    RandGaussianNoised,
    RandGaussianSmoothd,
    Rand2DElasticd,
    ToTensord,
)


class DRIVEDataModule(pl.LightningDataModule):
    def __init__(self,
                 data_dir: str,
                 batch_size: int = 64,
                 num_workers: int = 4,
                 train_val_split: float = 0.8,
                 ):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.train_val_split = train_val_split

    def get_train_transform(self):
        return Compose([
            LoadImaged(keys=["image", "label", "mask"], reader="ITKReader"),
            EnsureChannelFirstd(keys=["image", "label", "mask"]),
            ScaleIntensityd(keys=["label", "mask"]),
            MaskIntensityd(keys=["image"], mask_key="mask"),
            Resized(keys=["image", "label", "mask"], spatial_size=(512, 512)),
            # Rand2DElasticd(keys=["image", "label", "mask"], spacing=(50, 50), magnitude_range=(5, 15)),
            RandFlipd(keys=["image", "label", "mask"], prob=0.5),
            RandRotate90d(keys=["image", "label", "mask"], prob=0.5, spatial_axes=(0, 1)),
            RandGaussianNoised(keys=["image"], prob=0.5),
            # RandGaussianSmoothd(keys=["image"], prob=0.5),
            NormalizeIntensityd(keys=["image"]),
            AsDiscreted(keys="label", to_onehot=2, threshold=0.5),
            ToTensord(keys=["image", "label", "mask"]),
        ])

    def get_val_transform(self):
        return Compose([
            LoadImaged(keys=["image", "label", "mask"], reader="ITKReader"),
            EnsureChannelFirstd(keys=["image", "label", "mask"]),
            ScaleIntensityd(keys=["label", "mask"]),
            MaskIntensityd(keys=["image"], mask_key="mask"),
            Resized(keys=["image", "label", "mask"], spatial_size=(512, 512)),
            NormalizeIntensityd(keys=["image"]),
            AsDiscreted(keys="label", to_onehot=2, threshold=0.5),
            ToTensord(keys=["image", "label", "mask"]),
        ])

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            # Paths for training images and labels
            train_image_dir = os.path.join(self.data_dir, 'training', 'images')
            train_label_dir = os.path.join(self.data_dir, 'training', '1st_manual')
            train_mask_dir = os.path.join(self.data_dir, 'training', 'mask')

            # Create file lists
            train_image_files = sorted(
                [os.path.join(train_image_dir, f) for f in os.listdir(train_image_dir) if f.endswith('.tif')])
            train_label_files = sorted(
                [os.path.join(train_label_dir, f) for f in os.listdir(train_label_dir) if f.endswith('.gif')])
            train_mask_files = sorted(
                [os.path.join(train_mask_dir, f) for f in os.listdir(train_mask_dir) if f.endswith('.gif')])

            # Pairing images with labels
            train_files = [{"image": img, "label": lbl, "mask": msk} for img, lbl, msk in
                           zip(train_image_files, train_label_files, train_mask_files)]

            # Splitting dataset into training and validation sets
            train_len = int(len(train_files) * self.train_val_split)
            val_len = len(train_files) - train_len
            self.train_dataset, self.val_dataset = random_split(train_files, [train_len, val_len])

            # Applying transforms
            self.train_dataset = CacheDataset(self.train_dataset, transform=self.get_train_transform())
            self.val_dataset = CacheDataset(self.val_dataset, transform=self.get_val_transform())

        if stage == 'test' or stage is None:
            # Paths for test images
            test_image_dir = os.path.join(self.data_dir, 'test', 'images')
            test_label_dir = os.path.join(self.data_dir, 'test', '1st_manual')
            test_mask_dir = os.path.join(self.data_dir, 'test', 'mask')

            # Create file list
            test_image_files = sorted(
                [os.path.join(test_image_dir, f) for f in os.listdir(test_image_dir) if f.endswith('.tif')])
            test_label_files = sorted(
                [os.path.join(test_label_dir, f) for f in os.listdir(test_label_dir) if f.endswith('.gif')])
            test_mask_files = sorted(
                [os.path.join(test_mask_dir, f) for f in os.listdir(test_mask_dir) if f.endswith('.gif')])

            # Preparing test dataset
            test_files = [{"image": img, "label": lbl, "mask": msk} for img, lbl, msk in
                          zip(test_image_files, test_label_files, test_mask_files)]
            self.test_dataset = CacheDataset(test_files, transform=self.get_val_transform())

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size,
                          num_workers=self.num_workers, persistent_workers=True,
                          shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size,
                          num_workers=self.num_workers, persistent_workers=True)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size,
                          num_workers=self.num_workers, persistent_workers=True)

#### 模型定义

接下来，我们需要定义一个用于分割视网膜血管的模型。我们可以使用 `UNet` 模型，它是一个经典的用于医学影像分割的模型。

UNet 的核心设计理念是一个对称的 “U” 形结构，包含一个收缩（下采样）路径和一个扩展（上采样）路径。收缩路径通过一系列卷积和池化操作捕获图像的上下文信息，而扩展路径则通过上采样和卷积操作恢复图像的空间维度，使模型能够进行精确的定位。

在 MONAI 框架中，UNet 已经被实现，并且提供了易于使用的接口。

In [9]:
from monai.networks.nets import UNet
from monai.networks import one_hot
import lightning.pytorch as pl
from monai.losses import DiceLoss, FocalLoss, DiceFocalLoss
from monai.metrics import DiceMetric, MeanIoU

class SegmentationModule(pl.LightningModule):
    def __init__(self, 
                 learning_rate=0.01,
                 backbone='unet',
                 loss='dice',
                 optimizer='Adam',
                 accuracy='dice',
                 ):
        super().__init__()
        self.learning_rate = learning_rate
        
        if backbone.lower() == 'unet':
            self.model = UNet(
                spatial_dims=2,
                in_channels=3,
                out_channels=2,
                channels=(16, 32, 64, 128, 256),
                strides=(2, 2, 2, 2),
                num_res_units=2,
            )
        else:
            raise NotImplementedError(f'Backbone {backbone} not implemented. '
                                      f'Please choose from [unet].')    
        
        if loss.lower() == 'dice':
            self.loss = DiceLoss( softmax=True)
        elif loss.lower() == 'focal':
            self.loss = FocalLoss()
        elif loss.lower() == 'dicefocal':
            self.loss = DiceFocalLoss( softmax=True)
        else:
            raise NotImplementedError(f'Loss {loss} not implemented. '
                                      f'Please choose from [dice, focal, dicefocal].')
        
        if optimizer.lower() == 'adam':
            self.optimizer = torch.optim.Adam
        elif optimizer.lower() == 'sgd':
            self.optimizer = torch.optim.SGD
        else:
            raise NotImplementedError(f'Optimizer {optimizer} not implemented. '
                                      f'Please choose from [Adam, SGD].')
        
        if accuracy.lower() == 'dice':
            self.accuracy = DiceMetric(include_background=False, reduction='mean')
        elif accuracy.lower() == 'iou':
            self.accuracy = MeanIoU(include_background=False, reduction='mean')
        else:
            raise NotImplementedError(f'Accuracy {accuracy} not implemented. '
                                      f'Please choose from [dice, iou].')
        
    def forward(self, x):
        return self.model(x)
    
    def configure_optimizers(self):
        return self.optimizer(self.parameters(), lr=self.learning_rate)
    
    def prepare_batch(self, batch):
        return batch['image'], batch['label']
    
    def training_step(self, batch, batch_idx):
        x, y = self.prepare_batch(batch)
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        self.log('train_loss', loss.mean(), on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = self.prepare_batch(batch)
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        y_hat_onehot = one_hot(torch.argmax(y_hat, dim=1, keepdim=True), num_classes=2)
        acc = self.accuracy(y_hat_onehot, y)
        self.log('val_loss', loss.mean(), prog_bar=True)
        self.log('val_acc', acc.mean(), prog_bar=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = self.prepare_batch(batch)
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        y_hat_onehot = one_hot(torch.argmax(y_hat, dim=1, keepdim=True), num_classes=2)
        acc = self.accuracy(y_hat_onehot, y)
        self.log('test_loss', loss.mean(), on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('test_acc', acc.mean(), on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return {'test_loss': loss.mean(), 'test_acc': acc.mean()}
    

#### 训练模型

现在，我们可以使用 PyTorch Lightning 来训练模型了。我们将使用 `Trainer` 类来训练模型，它封装了训练过程中的大量细节，使得训练过程更加简单。

In [10]:
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.tuner import Tuner
from lightning.pytorch import seed_everything
import torch

seed_everything(42)

# Initialize data module and model
data_module = DRIVEDataModule(data_dir='data/DRIVE', batch_size=4, num_workers=4)
model = SegmentationModule(learning_rate=0.01)

callbacks = [
    # Early stopping callback to prevent overfitting
    EarlyStopping(monitor='val_loss', patience=5, mode='min'),
    # Model checkpoint callback to save the best model(s)
    ModelCheckpoint(monitor='val_loss', mode='min', save_top_k=5, filename='{epoch}-{val_loss:.3f}-{val_acc:.4f}')
]

# Initialize trainer
trainer = pl.Trainer(
    max_epochs=20,  # Number of epochs, if -1, runs indefinitely
    # precision='bf16',  # precision of training, default is 32-bit
    callbacks=callbacks,  # callbacks defined above
    accelerator='cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    # device to use for training
)

# Tune the model hyperparameters
tuner = Tuner(trainer)

# # Find the optimal batch size that maximizes GPU utilization
# tuner.scale_batch_size(model, data_module)

# # Find the "optimal" learning rate (BE CAREFUL)
# tuner.lr_find(model, data_module)

# Fit the model
trainer.fit(model=model, datamodule=data_module)

# Test the model
trainer.test(model=model, datamodule=data_module, ckpt_path="best")

Seed set to 42
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/rizhong/miniconda3/lib/python3.11/site-packages/monai/transforms/io/array.py:205: UserWarning: required package for reader ITKReader is not installed, or the version doesn't match requirement.
  warnings.warn(
Loading dataset: 100%|██████████| 4/4 [00:00<00:00, 54.84it/s]

  | Name  | Type     | Params
-----------------------------------
0 | model | UNet     | 1.6 M 
1 | loss  | DiceLoss | 0     
-----------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.503     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/rizhong/miniconda3/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:293: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.
Loading dataset: 100%|██████████| 20/20 [00:00<00:00, 52.85it/s]
Restoring states from the checkpoint path at /Users/rizhong/Documents/notes/20231115_LightningMONAITutorial/lightning_logs/version_16/checkpoints/epoch=18-val_loss=0.156-val_acc=0.7243.ckpt
Loaded model weights from the checkpoint at /Users/rizhong/Documents/notes/20231115_LightningMONAITutorial/lightning_logs/version_16/checkpoints/epoch=18-val_loss=0.156-val_acc=0.7243.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_acc_epoch       │    0.7304187417030334     │
│      test_loss_epoch      │    0.15252387523651123    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss_epoch': 0.15252387523651123,
  'test_acc_epoch': 0.7304187417030334}]